<a href="https://colab.research.google.com/github/yuvalofek/FrequentistML/blob/master/FreqML2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Frequentist ML Assignment 2 - Logistic Regression** 
Yuval Epstain Ofek
---


---



# **The provided Prompt:**

1. Read sections: 4.1, 4.3 (not 4.3.1 and 4.3.2), 4.4-4.4.2  and the following paper:
https://leon.bottou.org/publications/pdf/compstat-2010.pdf

2. Grab a binary classification dataset from UCI or other repository. Divide your data into roughly 80% train, 10% validation, 10% test. 
3. Implement logistic regression with stochastic gradient descent as the optimization algorithm.
4. Implement SGD without regularization and report your % correct on the test dataset.
5. Implement SGD with regularization, select the best lambda parameter using the validation dataset, and report your % correct on the test dataset.
6. Plot the likelihood function with respect to iterations for unregularized and regularized on one set of axes. Which one converges to a higher likelihood, and why?


Optional, advanced things to try:

- Implement SGD with the L-1 penalty and use it for feature selection (it is not that hard actually)
- Compare SGD to Newton-Raphson by plotting the likelihood of both on one set of axes and explain why they are different.

In [1]:
#Getting a dataset
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data

--2020-06-19 19:26:03--  https://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 191873 (187K) [application/x-httpd-php]
Saving to: ‘abalone.data’

abalone.data        100%[===================>] 187.38K   362KB/s    in 0.5s    

2020-06-19 19:26:05 (362 KB/s) - ‘abalone.data’ saved [191873/191873]



# Alabone Data Set
[Link to Dataset Page](https://archive.ics.uci.edu/ml/datasets/Abalone)

**Attribute Desctiption:**
* Sex / nominal / -- / M, F, and I (infant)
* Length / continuous / mm / Longest shell measurement
* Diameter / continuous / mm / perpendicular to length
* Height / continuous / mm / with meat in shell
* Whole weight / continuous / grams / whole abalone
* Shucked weight / continuous / grams / weight of meat
* Viscera weight / continuous / grams / gut weight (after bleeding)
* Shell weight / continuous / grams / after being dried
* Rings / integer / -- / +1.5 gives the age in years

Since the Sex is not binary I decided to omitt the infant. 

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [7]:
#copy of getdata from last assignment
def getdata (name, cols= None, shuffle = True ):
  # getdata (name, shuffle = True)
  # function to read the csv into a numpy array and shuffle columns. 
  # Shuffles columns if shuffle is true and leaves them if false

  # Loads the CSV data
  data = pd.read_csv(name, header=None, usecols=cols)
  #shuffling
  if shuffle:
    return data.sample(frac=1).reset_index(drop=True)
  else:
    return data

In [ ]:
#Extract from csv & shuffle
dshuf = getdata('abalone.data')


In [25]:
#Clean out the infant label
dshuf = dshuf[dshuf[0] != 'I']
#Assign M = TRUE, and F = FALSE, and check
print(dshuf[0])
dshuf[0] = dshuf[0] == 'M'
print(dshuf[0])

0       M
2       M
3       M
4       M
5       M
       ..
4170    M
4171    F
4172    M
4174    F
4176    M
Name: 0, Length: 2835, dtype: object
0        True
2        True
3        True
4        True
5        True
        ...  
4170     True
4171    False
4172     True
4174    False
4176     True
Name: 0, Length: 2835, dtype: bool


In [28]:
### Spliting into Training, Validation, and Test

#percentage of data for training is 80% and for testing is 10%
perTrain = 0.80
perTest = 0.10 
len = dshuf.shape[0]

#indices to split 
i_train =  round(len*perTrain)
i_test = round(len*(perTest+perTrain))

#actually splitting data
tr_d  = dshuf[:i_train]
val_d = dshuf[i_train:i_test]
test_d = dshuf[i_test:]

#display sizes of the split data to verify splitting was done correctly
print('Training shape: ' + str(tr_d.shape), 'Validation shape: ' +str(val_d.shape),'Test shape: ' + str(test_d.shape))

Training shape: (2268, 9) Validation shape: (284, 9) Test shape: (283, 9)


In [ ]:
def logSGD (x,y):
  